In [109]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt
from os.path import dirname, join as pjoin
from scipy import io as sio
#from keras.utils.np_utils import to_categorical

In [110]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import datasets
from tensorflow.keras import optimizers

In [111]:
num_classes = 26
batch_size = 2000
epochs = 20
neurons = 1000

mat_contents = sio.loadmat('ANN-Handwritten-Char-Recognition-master/matlab/emnist-letters.mat')
data = mat_contents['dataset']

X_train = data['train'][0,0]['images'][0,0]
y_train = data['train'][0,0]['labels'][0,0]
X_test = data['test'][0,0]['images'][0,0]
y_test = data['test'][0,0]['labels'][0,0]

val_start = X_train.shape[0] - X_test.shape[0]
X_val = X_train[val_start:X_train.shape[0],:]
y_val = y_train[val_start:X_train.shape[0]]
X_train = X_train[0:val_start,:]
y_train = y_train[0:val_start]

y_train -=1
y_val -=1
y_test -=1

#convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print(X_train.shape[0], "training samples")
print(X_val.shape[0], "validation samples")
print(X_test.shape[0], "test samples")
print(X_val.shape, y_val.shape)

104000 training samples
20800 validation samples
20800 test samples
(20800, 784) (20800, 26)


In [112]:
model = tf.keras.Sequential()
model.add(layers.Dense(neurons, activation='relu', input_shape=(784,)))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(neurons, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_54 (Dense)             (None, 1000)              785000    
_________________________________________________________________
dropout_36 (Dropout)         (None, 1000)              0         
_________________________________________________________________
dense_55 (Dense)             (None, 1000)              1001000   
_________________________________________________________________
dropout_37 (Dropout)         (None, 1000)              0         
_________________________________________________________________
dense_56 (Dense)             (None, 26)                26026     
Total params: 1,812,026
Trainable params: 1,812,026
Non-trainable params: 0
_________________________________________________________________


In [113]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size = batch_size,
                    epochs = epochs,
                    verbose = 1,
                    validation_data = (X_val, y_val))

Train on 104000 samples, validate on 20800 samples
Epoch 1/20
104000/104000 [==============================] - 5s 49us/sample - loss: 25.1851 - accuracy: 0.5053 - val_loss: 1.4425 - val_accuracy: 0.7067
Epoch 2/20
104000/104000 [==============================] - 5s 49us/sample - loss: 1.7533 - accuracy: 0.6494 - val_loss: 0.9943 - val_accuracy: 0.7601
Epoch 3/20
104000/104000 [==============================] - 5s 52us/sample - loss: 1.1599 - accuracy: 0.7311 - val_loss: 0.7186 - val_accuracy: 0.8029
Epoch 4/20
104000/104000 [==============================] - 6s 56us/sample - loss: 0.7906 - accuracy: 0.7862 - val_loss: 0.5711 - val_accuracy: 0.8312
Epoch 5/20
104000/104000 [==============================] - 6s 55us/sample - loss: 0.6291 - accuracy: 0.8201 - val_loss: 0.5041 - val_accuracy: 0.8533
Epoch 6/20
104000/104000 [==============================] - 6s 56us/sample - loss: 0.5426 - accuracy: 0.8403 - val_loss: 0.4547 - val_accuracy: 0.8684
Epoch 7/20
104000/104000 [================

In [114]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.4365356271742628
Test accuracy: 0.9001923


##### 20 epochs, 1000 batch size,  800 neurons - 89.69%
##### 20 epochs, 2000 batch size,  800 neurons - 89.56%
##### 20 epochs, 1000 batch size,  500 neurons - 88.58%
##### 20 epochs, 1000 batch size, 1000 neurons - 89.85%
##### 20 epochs, 2000 batch size, 1000 neurons - 90.25%
##### 20 epochs, 2000 batch size,  800 neurons - 89.74%
##### 20 epochs, 2000 batch size, 1200 neurons - 90.27%
##### 20 epochs, 2500 batch size, 1200 neurons - 89.64%
##### 15 epochs, 2000 batch size, 1200 neurons - 89.67%
##### 20 epochs, 1500 batch size, 1200 neurons - 89.56%
##### 20 epochs, 1500 batch size,  800 neurons - 89.57%
##### 20 epochs, 1000 batch size,  800 neurons - 89.18%
##### 20 epochs, 2000 batch size,  800 neurons - 89.81%
##### 20 epochs, 2000 batch size, 1200 neurons - 89.62%
##### 20 epochs, 2000 batch size, 1000 neurons - 90.02%


